In [1]:
import tensorflow as tf

In [43]:
import math

In [236]:
def arctanh(x):
    return tf.log(tf.divide(1+x,1-x))

Initialise all of the variables

In [237]:
def inner_prod(r_in, r_out, theta_in, theta_out):
    cosine = tf.cos(theta_in - theta_out)
    radius = tf.multiply(arctanh(r_in), arctanh(r_out))
    return 4 * tf.multiply(cosine, radius)

In [238]:
def tensor_inner_prod(r_example, r_sample, theta_example, theta_sample):
    r1 = arctanh(r_example)
    r2 = arctanh(r_sample)
    radius_term = r1[:, None] + r2[None, :]
    cos_term = theta_example[:, None] - theta_sample[None, :]
    return tf.squeeze(4* tf.multiply(cos_term, radius_term))

In [244]:
def nce_loss(true_logits, sampled_logits):
        true_xent = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.ones_like(true_logits), logits=true_logits)
        sampled_xent = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.zeros_like(sampled_logits), logits=sampled_logits)
        nce_loss_tensor = (tf.reduce_sum(true_xent) +
                           tf.reduce_sum(sampled_xent)) / 2
        return nce_loss_tensor

In [239]:
radius_in = tf.Variable(tf.sqrt(tf.random_uniform([5,1])))  # radius
# radius_out = tf.Variable(tf.zeros([5,1]))
radius_out = tf.Variable(tf.sqrt(tf.random_uniform([5,1])))
theta_in = tf.Variable(2*np.pi*tf.random_uniform([5,1]))  # angle
theta_out = tf.Variable(2*np.pi*tf.random_uniform([5,1]))
# theta_out = tf.Variable(tf.zeros([5,1]))
sm_b = tf.Variable(tf.zeros([5,1]))
examples = tf.Variable([1,2])
labels = tf.Variable([3,4])

In [240]:
example_radius = tf.nn.embedding_lookup(radius_in, examples)
example_theta = tf.nn.embedding_lookup(theta_in, examples)
true_radius = tf.nn.embedding_lookup(radius_out, labels)
true_theta = tf.nn.embedding_lookup(theta_out, labels)

In [241]:
sampled_ids = tf.Variable([0,1,2])
sampled_radius = tf.nn.embedding_lookup(radius_out, sampled_ids)
sampled_theta = tf.nn.embedding_lookup(theta_out, sampled_ids)
true_b = tf.nn.embedding_lookup(sm_b, labels)

In [242]:
true_logits = inner_prod(example_radius, true_radius, example_theta, true_theta) + true_b

In [243]:
sampled_b = tf.nn.embedding_lookup(sm_b, sampled_ids)
sampled_b_vec = tf.reshape(sampled_b, sampled_ids.get_shape())

In [228]:
sampled_logits = tensor_inner_prod(example_radius, sampled_radius, example_theta, sampled_theta) + sampled_b_vec

In [215]:
r1 = arctanh(example_radius)
r2 = arctanh(sampled_radius)
radius_term = r1[:, None] + r2[None, :]
cos_term = example_theta[:, None] - sampled_theta[None, :]
retval = 4* tf.multiply(cos_term, radius_term)

In [245]:
loss = nce_loss(true_logits, sampled_logits)

In [246]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
# print(sess.run(sampled_radius))
# print(sess.run(r1))
# print(sess.run(r2))
# print(sess.run(radius_term))
# print(sess.run(cos_term))
# print(sess.run(retval))
# print(sess.run(sampled_logits))
print(sess.run(true_logits))
print(sess.run(loss))

[[ -2.73551893]
 [ 13.55606556]]
87.678


In [217]:
r1.get_shape()

TensorShape([Dimension(2), Dimension(1)])

In [224]:
tf.squeeze(example_radius).get_shape()

TensorShape([Dimension(2)])

In [219]:
retval.get_shape()

TensorShape([Dimension(2), Dimension(3), Dimension(1)])

In [229]:
sampled_logits.get_shape()

TensorShape([Dimension(2), Dimension(3)])

In [250]:
x1 = np.array([1,2])
x2 = np.array([3,4])
x3 = np.concatenate((x1.T,x2.T), axis=1)
x3

IndexError: axis 1 out of bounds [0, 1)